In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

In [ ]:
train = pd.read_csv('Devex_train.csv',encoding='latin-1')


In [ ]:
df = train.copy()

In [ ]:
onehotencode = df[['Label 1', 'Label 2', 'Label 3',\
    'Label 4', 'Label 5', 'Label 6',\
    'Label 7','Label 8', 'Label 9', \
    'Label 10', 'Label 11', 'Label 12']].applymap(str).applymap(lambda x: x.split('-')[0].strip())

In [ ]:
def to_frequency_table(data):
    frequencytable = {}
    for key in data:
        #print(key)
        for element in data[key]:
            if element in frequencytable:
                #print("Hello")
                #print(key)
                frequencytable[element] += 1
            else:
                frequencytable[element] = 1
    return frequencytable
D = to_frequency_table(onehotencode)
del D['nan']
D

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def hot_encode_free_text(df,col_list):
    df1 = df[col_list].apply(lambda x: ' '.join(x), axis=1)
    #cv = CountVectorizer(vocabulary=vocab,ngram_range=(1, 2))#token_pattern=r'(?u)\b\w+\b
    #cv = CountVectorizer(token_pattern=r'(?u)\b\w+\b')#\b[^\d\W]+\b
    cv = CountVectorizer(tokenizer=my_tokenizer)
    X_train = cv.fit_transform(df)
    X_train = pd.DataFrame(X_train.toarray(), columns=cv.get_feature_names())
    return X_train.applymap(convert_num)
    #return X_train
def convert_num(num):
    if num == 0:
        return 0
    else:
        return 1

In [ ]:
#\b[^\d\W]+\b
import spacy
#from html import unescape
#from spacy.lang.en.examples import sentences
# create a spaCy tokenizer
spacy.load('en')
lemmatizer = spacy.lang.en.English()
def my_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens])
#my_tokenizer(vocab[0])

In [ ]:
vocab = ['3.1.1','3.1.2',
 '3.2.1','3.2.2','3.3.1',
'3.3.2','3.3.3','3.3.4',
'3.3.5','3.4.1','3.4.2',
'3.5.1','3.5.2','3.6.1',
'3.7.1','3.7.2','3.8.1',
'3.8.2','3.9.1','3.9.2',
'3.9.3','3.a.1','3.b.1',
'3.b.2','3.b.3','3.c.1',
 '3.d.1']

In [ ]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 5000

In [ ]:
hot_encoded_labels = hot_encode_free_text(onehotencode, onehotencode.columns.tolist())

In [ ]:
data = pd.merge(train[['Unique ID', 'Type', 'Text']], hot_encoded_labels,\
                         left_on=train[['Unique ID', 'Type', 'Text']].index,\
                         right_on=hot_encoded_labels.index, how ='inner')


In [ ]:
import re

#Without removing stop words

def clean_text_final(text):
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [ ]:
for i in range(len(data)):
    data.at[i,'Text'] = clean_text_final(data.at[i,'Text'])

In [ ]:
df1 = data.rename(columns={'Unique ID':'ID'})

In [ ]:
df2 = df1.drop(['Type'], axis=1)